## 1 - Load the data and Preprocessing

In [18]:
dataset_dir = 'C:/Users/himan/ML/PycharmProjects/DeepLearning/Yulia_classification/data/'
IMG_SIZE = 96

In [ ]:
# Data Preparation

categories = ['defocused_blurred', 'motion_blurred','sharp']
training_bgr = []
training_rgb = []

# saving data with respect to RGB and BGR
def create_training_data():
    for i in range(0,2,1):
        print(i)
        for category in categories:
            path = dataset_dir+category
            print(path)
            class_num = categories.index(category)

            for img in os.listdir(path):
                if i == 0:
                    img_array = cv2.imread(os.path.join(path, img))
                    img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                    training_bgr.append([img_array, class_num])
                if i==1:
                    img_array = cv2.imread(os.path.join(path, img))  
                    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
                    img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                    training_rgb.append([img_array, class_num])
                    
        i+=1
                    
create_training_data()

In [ ]:
# Data Preprocessing

def preprocessing(training, n):
    # shuffling the dataset
    random.shuffle(training)

    # assigning labels and features

    X = []
    y = []
    for features, label in training:
        X.append(features)
        y.append(label)

    # resizing features in accordance with CNN
    X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

    # Normalising X and converting labels to categorical features
    X = X.astype('float32')
    X /= 255

    y = np_utils.to_categorical(y,n)

    # splitting X and y for use in CNN
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)
    
    return X_train, X_test, y_train, y_test

Each aforementioned class has three types of data with the class labels,  thus data for each of the three classes has the following images:
* Original Images from the link
* Rotated Images
* BGR and RGB Images

### Rotate

In [ ]:
angles = [0, 270, 90, 180] # (0 degree, 90 degree clockwise, 90 degree anticlockwise, 180 degree or flipped image)

def rotate(image, angle):
    image = imutils.rotate(image, angle=angle) # this rotates image in anti-clockwise direction
    return image

training_rotate = []

for i in range(len(training_rgb)):
    for angle in angles:
        class_num = angles.index(angle)
        training_rgb[i][0] = rotate(training_rgb[i][0], angle)
        training_rotate.append([training_rgb[i][0], class_num]) 

for i in range(len(training_bgr)):
    for angle in angles:
        class_num = angles.index(angle)
        training_bgr[i][0] = rotate(training_bgr[i][0], angle)
        training_rotate.append([training_bgr[i][0], class_num])  

X_train_rotate, X_test_rotate, y_train_rotate, y_test_rotate = preprocessing(training_rotate, len(angles))
len(training_rotate)

### Quality

In [ ]:
training_quality = []

for i in range(len(training_rgb)):
    for angle in angles:
        training_rgb[i][0] = rotate(training_rgb[i][0], angle)
        training_quality.append([training_rgb[i][0], training_rgb[i][1]]) 
        
for i in range(len(training_bgr)):
    for angle in angles:
        training_bgr[i][0] = rotate(training_bgr[i][0], angle)
        training_quality.append([training_bgr[i][0], training_bgr[i][1]])  
         

X_train_quality, X_test_quality, y_train_quality, y_test_quality = preprocessing(training_quality, len(categories))
len(training_quality)

### Model

In [ ]:
mode = ['RGB', 'BGR']
training_mode = []

for i in range(len(training_rgb)):
    for angle in angles:
        training_rgb[i][0] = rotate(training_rgb[i][0], angle)
        training_mode.append([training_rgb[i][0], mode.index('RGB')])
        
for i in range(len(training_bgr)):
    for angle in angles:
        training_bgr[i][0] = rotate(training_bgr[i][0], angle)
        training_mode.append([training_bgr[i][0], mode.index('BGR')])  

X_train_mode, X_test_mode, y_train_mode, y_test_mode = preprocessing(training_mode, len(mode))
len(training_mode)

## 2 - Models
Write a code for training, validating, testing for all the models you are capable of in the notebook `model.ipynb`. Put as much comments as you can what you do, how the pre-trained model works, which issues you face. We want to deploy your model into production, please help us to do so by clearly saying what you do and why it is good.

**Write a clear conclusion which model can be recommended to the client at the of this part.**

In [ ]:
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='loss', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

A **Squeeze and Excitation(SE)** block consists of two operations: 
* A squeeze operation and an excitation operation. The squeeze operation is a global pooling operation that reduces the spatial dimensions of the feature maps produced by the previous convolutional layer to a single channel. 
* The excitation operation then applies a set of fully connected layers to this single-channel representation, which produces a set of weights that are applied to the original feature maps to emphasize or suppress specific channels.

In [ ]:
def SE_block(inputs, ratio=8):
    b, h, w, c = inputs.shape
    # squeeze 
    x = GlobalAveragePooling2D()(inputs)
    x = Reshape((1, 1, c))(x)
    # extraction
    x = Dense(c//ratio, activation='relu', kernel_initializer='glorot_uniform', use_bias=False)(x)
    x = Dense(c, activation='sigmoid', kernel_initializer='glorot_uniform', use_bias=False)(x)

    # scaling
    x = Multiply()([inputs, x]) # x*inputs

    return x

# its returns tensor with original input shape

inputs = Input(shape=(128, 128, 32))
print(SE_block(inputs).shape)

def create_model(name):

    input_tensor = Input(shape=[IMG_SIZE, IMG_SIZE, 3], name=name)
    x = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(input_tensor)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    
    x = SE_block(x)

    x = Conv2D(filters=64, kernel_size=(3, 3), padding='same',activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = SE_block(x)

    x = Conv2D(filters=64, kernel_size=(3, 3), padding='same',activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    
    # fully connected
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    output_tensor = Dense(9, activation='relu')(x)
    model = Model(inputs=[input_tensor], outputs=[output_tensor])
    
    return model

# fittin the model
# borrowed from Yulia

model_quality = create_model('class1')
model_rotate = create_model('class2')
model_mode = create_model('class3')

# segregating out ouput layers for three different classes
mergedOutput = concatenate([model_quality.output, model_rotate.output, model_mode.output])
out_quality = Dense(3, activation='softmax', name='quality')(mergedOutput)
out_rotate = Dense(4, activation='softmax', name='rotate')(mergedOutput)
out_mode = Dense(2, activation='softmax', name='mode')(mergedOutput)

merged_model = Model(inputs=[model_quality.input, model_rotate.input, model_mode.input],
                     outputs=[out_quality, out_rotate, out_mode])

print(merged_model.summary())

merged_model.compile(optimizer='adam',
                     loss='categorical_crossentropy',
                     metrics="categorical_accuracy") 

history = merged_model.fit(x={'class1': X_train_quality, 'class2': X_train_rotate, 'class3': X_train_mode},
                 y={'quality': y_train_quality, 'rotate': y_train_rotate, 'mode': y_train_mode},
                 batch_size=64, epochs=25, callbacks=[learning_rate_reduction])

## 3 - Saving the model

In [ ]:
# saving the model

# serialize model to JSON
model_json = merged_model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
merged_model.save_weights("model.h5")
print("Saved model to disk")

## 4 - Conclusion

* Accuracy regarding to Mode and Rotation maxed out before 4 epochs since we are using SE blocks, which extracts information from channels.
* For the second and third Classes we can definitely use SE blocks
* Regarding the quality class we can definitely do better with better data quality.